In [ ]:
import json
import pandas as pd
import re

import sys
sys.path.append('lib')
from cleaning_helpers import remove_null_items_recursive

### pull data

In [ ]:
data = {}
with open('intermediate_outputs/parsed_data.json', 'r') as f:
    data = json.load(f)

In [ ]:
df = pd.DataFrame(data.values())
df.head() 

In [ ]:
with open("debug_files/debug_raw_component_relations.json", "w") as f:
    json.dump(remove_null_items_recursive({
        k: { 
            'dependencies': v.get('dependencies'),
            'triggers': v.get('triggers'), 
            'events': v.get('events') 
        } 
        for k,v 
        in data.items()
    }), f, indent=2 * " ")

#### test

### define some variables for use below

In [ ]:
code_snippets = {
    key: " | ".join(filter_triggers_from_code_snippets(value["component_relations"]["dependency_code_snippets"]))
    for key, value 
    in data.items() 
    if len(value["component_relations"]["dependency_code_snippets"]) > 0
}

triggers = {
    key: {
        trigger_key: {
            "event": "code_trigger",
            "type": data.get(trigger_key, {}).get("type"),
            "method": "trigger",
            "pluginId": trigger_key
        }
        for trigger_key
        in set([
            re.sub(".trigger", "", trigger_string)
            for trigger_string
            in value["component_relations"]["trigger_code_snippets"]
        ])
    }
    for key, value 
    in data.items() 
    if len(value["component_relations"]["trigger_code_snippets"]) > 0
}

### split events into triggers and non-triggers

In [ ]:
non_trigger_events = {}

for key, value in data.items():
    for event in value.get("events", []):
        event_key = event.get("pluginId")
        if (event_key != None and event_key != ""):
            if event.get("method") == "trigger":
                if not key in triggers:
                    triggers[key] = {}
                triggers[key][event_key] = event
            else:
                if not key in non_trigger_events:
                    non_trigger_events[key] = {}
                non_trigger_events[key][event_key] = event

In [ ]:
with open("debug_files/debug_triggers.json", 'w') as f:
    json.dump(triggers, f, indent=2 * " ")

##### get dependency list -- search code_snippets for component-name + '.'

In [ ]:
dependencies = {
    parent_key: [
        dependent_key
        for dependent_key
        in data.keys()
        if f"{dependent_key}." in snippets
    ] 
    for parent_key, snippets 
    in code_snippets.items()
}

for key, value in dependencies.items():
    data[key]["component_relations"]["dependencies"] = value
remove_null_items_recursive(dependencies)

### precompute reverse relations : 
#### grab a list of all components that depend on a given component
- e.g this component updates this list of components

In [ ]:
# reverse dependencies map into dependants map
for parent in dependencies.keys():
    if not "component_relations" in data[parent]:
        data[parent]["component_relations"] = {}
    if not "dependants" in data[parent]["component_relations"]:
        data[parent]["component_relations"]["dependants"] = []
    for dependant, dependency_components in dependencies.items():
        # if parent is a dependency of dependant
        if parent in dependency_components:
            data[parent]["component_relations"]["dependants"].append({
                "event": "on_value_change",
                "type": data[dependant].get("type"),
                "method": "update",
                "pluginId": dependant,
                "triggered_by": parent,
                "derived_event": True
            })

In [ ]:
def add_events_to_dependants_list(events_map: dict):
    global data;
    for parent, events in events_map.items():
        if not "component_relations" in data[parent]:
            data[parent]["component_relations"] = {}
        if not "dependants" in data[parent]["component_relations"]:
            data[parent]["component_relations"]["dependants"] = []
            
        for event in events.values():
            event["triggered_by"] = parent
            data[parent]["component_relations"]["dependants"].append(event)

In [ ]:
add_events_to_dependants_list(triggers)
add_events_to_dependants_list(non_trigger_events)

In [ ]:
with open("debug_files/debug_dependencies.json", "w") as f:
    json.dump({
        k: {
            'component_relations': v.get('component_relations'),
            'events': v.get('events')
        }
        for k,v 
        in data.items()
    }, f, indent=2 * " ")

#### intermediate output files

In [ ]:
dependency_graph = {key: value["component_relations"] for key, value in data.items()}
with open("debug_files/filtered_dependency_graph.json", "w") as f:
    json.dump(remove_null_items_recursive(dependency_graph), f, indent=2 * " ")

In [ ]:
with open("intermediate_outputs/dependency_mapped_data.json", 'w') as f:
    f.write(json.dumps(data, indent=2 * " "))